In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as img
import numpy as np
import cv2
import os, shutil
from scipy.spatial import distance
import xml.etree.ElementTree as ET

### Annotation 파일 읽어오기
https://codechacha.com/ko/python-parsing-xml-and-pretty-print/

In [2]:
import os 

path = './total_20220724_20220818_update'
annotation_list = os.listdir(path) 
annotation_list.remove('images')
print(annotation_list)

['annotations59.xml', 'annotations16.xml', 'annotations4.xml', 'annotations40.xml', 'annotations11.xml', 'annotations34.xml', 'annotations12.xml', 'annotations2.xml', 'annotations25.xml', 'annotations62.xml', 'annotations42.xml', 'annotations13.xml', 'annotations54.xml', 'annotations51.xml', 'annotations43.xml', 'annotations63.xml', 'annotations19.xml', 'annotations50.xml', 'annotations46.xml', 'annotations44.xml', 'annotations5.xml', 'annotations55.xml', 'annotations31.xml', 'annotations36.xml', 'annotations38.xml', 'annotations39.xml', 'annotations14.xml', 'annotations60.xml', 'annotations47.xml', 'annotations21.xml', 'annotations26.xml', 'annotations65.xml', 'annotations58.xml', 'annotations57.xml', 'annotations28.xml', 'annotations9.xml', 'annotations64.xml', 'annotations32.xml', 'annotations33.xml', 'annotations30.xml', 'annotations15.xml', 'annotations24.xml', 'annotations48.xml', 'annotations23.xml', 'annotations18.xml', 'annotations22.xml', 'annotations10.xml', 'annotations61.x

In [3]:
def get_occupied_info(polyline, width, height):
    label = polyline.get('label')
    points = polyline.get('points') # 2개
    bb_points, _ = extract_bb_points(points, width, height)
    return label, bb_points

def get_empty_info(polygon, width, height):
    label = polygon.get('label')
    points = polygon.get('points') # 4개 or 5개
    bb_points, ps = extract_bb_points(points, width, height)
    poly_points = ps
    empty_type = polygon.find('attribute').text
    return label, bb_points, poly_points, empty_type
    
def get_obstacle_info(polygon, width, height):
    label = polygon.get('label')
    points = polygon.get('points')
    bb_point, _ = extract_bb_points(points, width, height)
    obs_type = polygon.find('attribute').text
    return label, bb_points, obs_type

# 각 행의 n 번째 열을 기준으로 sort (오름차순)
# x : nd array
# axis : 기준 열
def sort_ndarray_by_col(x, axis=0):
    size = x.shape[0]
    col = [x[i][axis] for i in range(size)] # 기준 행의 원소들을 추출
    sorted_col = sorted(col) # 추출된 원소들을 정렬
    rank = [sorted_col.index(x[i][axis]) for i in range(size)] # 정렬된 index 추출
    #print(rank)
    try:
        sorted_array = [x[rank.index(i)] for i in range(size)]  # 정렬된 index 순서대로 정렬
    except: # 똑같은 순위가 있는 경우 예외처리
        overlap_rank = []
        for i in range(size):
            if rank.count(i) > 1:
                overlap_rank.append(i)
        for r in overlap_rank:
            overlap_list = [i for i in range(size) if rank[i]==r]
            for k, idx in enumerate(overlap_list):
                rank[idx] += k
        sorted_array = [x[rank.index(i)] for i in range(size)]  # 정렬된 index 순서대로 정렬

    return np.array(sorted_array)

# 함수화
def perspective_transform(src, point, height=256, width=256):    
    # 4 꼭지점 분류 
    sorted_by_y= sort_ndarray_by_col(point, 1)
    top_two = sorted_by_y[0:2]
    topLeft, topRight = sort_ndarray_by_col(top_two, 0)
    bottom_two = sorted_by_y[2:4]
    bottomLeft, bottomRight = sort_ndarray_by_col(bottom_two, 0)
    #print("tl:", topLeft)
    #print("bl", bottomLeft)
    #print("tr", topRight)
    #print("br", bottomRight)
    
    
    # 변환 전 4개 좌표 
    srcPoint = np.float32([topLeft, topRight, bottomRight , bottomLeft])

    # 변환 후 4개 좌표
    dstPoint = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

    # Perspective transformation
    matrix = cv2.getPerspectiveTransform(srcPoint, dstPoint)
    dst = cv2.warpPerspective(src, matrix, (width, height))
    
    return dst

# 이미지 상에 polygon 그리기
def draw_label(img_path, data, save_path=""):
    '''
    image = img.imread(img_path)
    plt.imshow(image)

    ax = plt.gca()

    point = np.array([[data['p1'][0], data['p1'][1]],
                      [data['p2'][0], data['p2'][1]],
                      [data['p3'][0], data['p3'][1]],
                      [data['p4'][0], data['p4'][1]]])
    #print(point)

    polygon = patches.Polygon(point, fill=None ,edgecolor='k',ls='solid',lw=3)

    ax.add_patch(polygon)

    plt.show()
    '''
    src = cv2.imread(img_path, cv2.IMREAD_COLOR)
    point = np.array([[data['b1'][0], data['b1'][1]],
                      [data['b2'][0], data['b2'][1]],
                      [data['b3'][0], data['b3'][1]],
                      [data['b4'][0], data['b4'][1]]], dtype=np.float32)
    
    dst = cv2.polylines(src, np.int32([point]), True, (255, 0, 0), 6)

    return dst

def extract_bb_points(points, width, height):
    if len(points.split(';')) == 5: # 라벨링된 point가 5개인 경우
        ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
             (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1])),
             (float(points.split(';')[2].split(',')[0]), float(points.split(';')[2].split(',')[1])),
             (float(points.split(';')[3].split(',')[0]), float(points.split(';')[3].split(',')[1])),
             (float(points.split(';')[4].split(',')[0]), float(points.split(';')[4].split(',')[1]))]
        left, right, up, down = [], [], [], []
        for i, p in enumerate(ps):
            #print(i,p)
            if p[0] == 0: # 점이 이미지 왼쪽 끝에 존재
                left.append(i)
            if p[0] == float(width): # 점이 이미지 오른쪽 끝에 존재
                right.append(i)
            if p[1] == 0: # 점이 이미지 위쪽 끝에 존재
                up.append(i)
            if p[1] == float(height): # 점이 이미지 아래쪽 끝에 존재
                down.append(i)
        #print(len(left), len(right), len(up), len(down))

        if len(left) == 2: # 이미지 왼쪽 끝에 점이 두 개 있는 경우 하나 제거
            del ps[left[0]]
        elif len(right) == 2: # 이미지 오른쪽 끝에 점이 두 개 있는 경우 하나 제거
            del ps[right[0]]
        elif len(up) == 2: # 이미지 위쪽 끝에 점이 두 개 있는 경우 하나 제거
            del ps[up[0]]
        elif len(down) == 2: # 이미지 아래쪽 끝에 점이 두 개 있는 경우 하나 제거
            del ps[down[0]]
        
        # 4 꼭지점 분류 
        sorted_by_y= sort_ndarray_by_col(np.array(ps), 1)
        top_two = sorted_by_y[0:2]
        topLeft, topRight = sort_ndarray_by_col(top_two, 0)
        bottom_two = sorted_by_y[2:4]
        bottomLeft, bottomRight = list(sort_ndarray_by_col(bottom_two, 0))

    elif len(points.split(';')) == 4: # 라벨링된 point가 4개인 경우 = polygon 라벨링 (없음)
        ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
             (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1])),
             (float(points.split(';')[2].split(',')[0]), float(points.split(';')[2].split(',')[1])),
             (float(points.split(';')[3].split(',')[0]), float(points.split(';')[3].split(',')[1]))]

        # 4 꼭지점 분류 
        sorted_by_y= sort_ndarray_by_col(np.array(ps), 1)
        top_two = sorted_by_y[0:2]
        topLeft, topRight = sort_ndarray_by_col(top_two, 0)
        bottom_two = sorted_by_y[2:4]
        bottomLeft, bottomRight = list(sort_ndarray_by_col(bottom_two, 0))

    elif len(points.split(';')) == 2: # 라벨링된 point가 2개인 경우 = 직선 라벨링 (있음))
        ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
             (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1]))]

        # 2 꼭지점 분류
        if ps[0][0] < ps[1][0]:
            bottomLeft = ps[0]
            bottomRight = ps[1]
        else:
            bottomLeft = ps[1]
            bottomRight = ps[0]
    else:
        return None, None
    
    print(len(points.split(';')))
    print('bb', bottomLeft, bottomRight) # bounding box 생성에 기준이 되는 주차면 앞 라인의 두 점
                    
    # 라인으로부터 바운딩 박스 생성
    x_offset = 0
    width = distance.euclidean(bottomLeft, bottomRight)+x_offset
    y_offset = width/10.0
    height = -width*2/3.0
    #print(ps, width, height)


    b1 = (bottomLeft[0], bottomLeft[1]+y_offset)
    b2 = (bottomRight[0], bottomRight[1]+y_offset)
    b3 = (b2[0], b2[1]+height)
    b4 = (b1[0], b1[1]+height)
    
    return [b1, b2, b3, b4], ps

In [4]:
occupied_cnt, empty_cnt, obstacle_cnt = 0, 0, 0

datasets = {}
datasets['occupied'] = {}
datasets['empty'] = {}
datasets['obstacle'] = {}

for annotation in annotation_list:
    print(annotation)
    # annotation 파일 불러오기
    filePath = os.path.join(path, annotation)
    #print(filePath)
    tree = ET.parse(filePath)
    root = tree.getroot()
    for image in root.findall("image"):
        image_attrib = image.attrib
        image_name = image_attrib.get('name')
        print(image_name)
        image_witdh = image_attrib.get('width')
        image_height = image_attrib.get('height')
        polylines = image.findall('polyline') # 있음 라벨링
        polygons = image.findall('polygon') # 없음 라벨링
        
        for polyline in polylines: # 있음
            polyline_label, bb_points = get_occupied_info(polyline, image_witdh, image_height)
            print(polyline_label) 
            occupied_cnt += 1 
            datasets['occupied'][str(occupied_cnt)]={'label':polyline_label, 'src':image_name,
                                                     'b1':bb_points[0],
                                                     'b2':bb_points[1],
                                                     'b3':bb_points[2],
                                                     'b4':bb_points[3]
                                                    }
            
        for polygon in polygons:
            label = polygon.get('label')
            if label == '없음': # 없음
                empty_label, bb_points, poly_points, empty_type = get_empty_info(polygon, image_witdh, image_height)
                print(empty_label, empty_type)
                if bb_points != None: # point 정보가 없는 경우 넘어감
                    empty_cnt += 1
                    datasets['empty'][str(empty_cnt)]={'label':empty_label, 'src':image_name, 
                                                       'type':empty_type,
                                                       'b1':bb_points[0],
                                                       'b2':bb_points[1],
                                                       'b3':bb_points[2],
                                                       'b4':bb_points[3],
                                                       'p1':poly_points[0],
                                                       'p2':poly_points[1],
                                                       'p3':poly_points[2],
                                                       'p4':poly_points[3],
                                                      }
                
            elif label == '장애물': # 장애물
                obs_label, bb_points, obs_type = get_obstacle_info(polygon, image_witdh, image_height)
                print(obs_label, obs_type)
                obstacle_cnt += 1
                datasets['obstacle'][str(obstacle_cnt)]={'label':obs_label, 'src':image_name,
                                                         'type':obs_type,
                                                         'b1':bb_points[0],
                                                         'b2':bb_points[1],
                                                         'b3':bb_points[2],
                                                         'b4':bb_points[3]
                                                        }
                
        #occupied_cnt += len(polylines)
        #empty_cnt += len(polygons)
        #print(image_name, '| 있음', len(polylines), '없음', len(polygons))
    

annotations59.xml
DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
2
bb (2229.14, 1607.28) (2970.92, 1617.87)
있음
2
bb (1210.07, 1527.8) (1999.54, 1531.33)
있음
2
bb (-0.0, 1510.37) (664.33, 1540.16)
있음
DJI_0001_001.MP4_snapshot_00.18_[2022.08.22_15.41.09].jpg
2
bb (1854.71, 1533.1) (2584.13, 1527.8)
있음
2
bb (360.55, 1607.28) (1277.18, 1594.91)
있음
2
bb (2743.09, 1568.42) (3403.63, 1545.46)
있음
DJI_0001_001.MP4_snapshot_00.20_[2022.08.22_15.41.12].jpg
2
bb (1563.3, 1589.62) (2303.32, 1582.55)
있음
2
bb (2665.38, 1529.57) (3278.23, 1524.27)
있음
2
bb (411.77, 1630.24) (1344.3, 1628.47)
있음
DJI_0001_001.MP4_snapshot_00.21_[2022.08.22_15.41.13].jpg
2
bb (1598.62, 1637.3) (2342.17, 1621.41)
있음
2
bb (2508.19, 1580.79) (3147.54, 1570.19)
있음
2
bb (291.67, 1564.89) (1157.09, 1563.12)
있음
DJI_0001_001.MP4_snapshot_00.22_[2022.08.22_15.41.15].jpg
2
bb (1605.69, 1573.72) (2282.12, 1568.42)
있음
2
bb (2605.33, 1591.38) (3382.43, 1568.42)
있음
2
bb (251.05, 1566.66) (1273.65, 1564.89)
있음
DJI_0001_001.MP4

DJI_0009_001.MP4_snapshot_01.40_[2022.08.18_15.05.10].jpg
2
bb (2513.49, 1494.24) (3237.61, 1398.87)
있음
2
bb (418.84, 1598.45) (1485.59, 1517.2)
있음
4
bb [1791.13 1195.76] [2010.14 1160.44]
장애물 이륜차
DJI_0009_001.MP4_snapshot_01.43_[2022.08.18_15.05.22].jpg
2
bb (477.12, 1473.05) (1377.85, 1467.75)
있음
2
bb (2345.7, 1492.48) (3442.48, 1469.52)
있음
4
bb [1407.88 1448.32] [2320.98 1432.43]
없음 일반형
DJI_0009_001.MP4_snapshot_01.47_[2022.08.18_15.05.27].jpg
2
bb (2377.49, 1473.05) (3332.98, 1428.9)
있음
4
bb [1218.9  1118.05] [1589.79 1105.69]
장애물 이륜차
4
bb [1430.84 1483.65] [2366.9  1455.39]
없음 일반형
4
bb [1088.21 1455.39] [1328.4 1183.4]
장애물 이륜차
DJI_0009_001.MP4_snapshot_01.49_[2022.08.18_15.05.30].jpg
4
bb [1112.93 1326.46] [1416.71 1132.18]
장애물 이륜차
4
bb [1524.44 1277.01] [1660.44 1093.33]
장애물 이륜차
4
bb [2564.71 1222.26] [3115.7 1347.7]
장애물 이륜차
4
bb [1750.5 1284.1] [1946.55 1206.36]
장애물 이륜차
4
bb [1939.49 1172.8 ] [2105.51 1188.7 ]
장애물 이륜차
4
bb [ 680.23 1245.22] [1054.7 1259.3]
장애물 이륜차
DJI_0009_001.M

VID_20220724_163250_00_013_2022-08-08_12-29-55_스크린샷.jpg
2
bb (583.79, 924.57) (923.78, 903.38)
있음
2
bb (1246.1, 845.1) (1443.03, 793.88)
있음
2
bb (1449.21, 790.35) (1577.25, 753.26)
있음
4
bb [225.27 930.75] [572.3 929. ]
없음 일반형
4
bb [948.5  903.38] [1246.1  845.1]
없음 일반형
VID_20220724_163250_00_013_2022-08-08_12-30-08_스크린샷.jpg
2
bb (98.99, 816.84) (276.48, 865.41)
있음
2
bb (1496.89, 954.6) (1888.09, 889.25)
있음
2
bb (659.74, 947.53) (1020.03, 969.61)
있음
4
bb [1035.93  970.49] [1475.7  954.6]
없음 일반형
VID_20220724_163250_00_013_2022-08-08_12-30-15_스크린샷.jpg
2
bb (1678.81, 869.82) (1886.2, 827.3)
있음
2
bb (1368.85, 921.92) (1632.89, 876.0)
있음
2
bb (5.38, 823.9) (117.53, 856.58)
있음
4
bb [131.66 853.04] [310.04 885.72]
없음 일반형
4
bb [973.23 938.7 ] [1354.72  924.57]
없음 일반형
VID_20220724_163250_00_013_2022-08-08_12-30-18_스크린샷.jpg
2
bb (689.76, 953.71) (1067.72, 966.96)
있음
2
bb (350.66, 905.14) (658.86, 951.95)
있음
4
bb [1111.87  959.01] [1502.19  933.4 ]
없음 일반형
4
bb [1524.27  936.05] [1830.7   875.12]
없

5
bb [2126.7  1540.16] [3083.95 1654.96]
없음 일반형
4
bb [1478.53 1254.05] [1867.8 1246.1]
없음 일반형
4
bb [1024.62 1261.11] [1423.77 1261.11]
없음 일반형
5
bb [   0.   1638.33] [ 819.75 1700.88]
없음 일반형
4
bb [ 883.33 1697.35] [1874.14 1672.62]
없음 일반형
DJI_0006_003.MP4_snapshot_01.20_[2022.08.22_12.39.18].jpg
2
bb (477.12, 1522.5) (1130.59, 1531.33)
있음
4
bb [1600.39 1254.05] [1973.05 1248.75]
없음 일반형
4
bb [2157.43 1673.51] [3095.26 1669.98]
없음 일반형
4
bb [1132.36 1674.39] [2070.19 1674.39]
없음 일반형
4
bb [2564.71 1254.05] [2912.64 1248.75]
없음 일반형
4
bb [2028.5  1249.63] [2425.89 1256.7 ]
없음 일반형
DJI_0006_003.MP4_snapshot_01.21_[2022.08.22_12.39.20].jpg
2
bb (436.5, 1614.34) (1135.89, 1601.98)
있음
2
bb (1633.95, 1554.29) (2299.78, 1541.93)
있음
4
bb [2527.62 1656.73] [3400.09 1649.66]
없음 일반형
4
bb [3507.83 1649.66] [3832.8  1487.18]
없음 일반형
4
bb [2585.9  1241.68] [2972.69 1241.68]
없음 일반형
4
bb [3105.15 1241.68] [3479.57 1239.92]
없음 일반형
DJI_0006_003.MP4_snapshot_01.22_[2022.08.22_12.39.21].jpg
2
bb (2554.11, 1540.16

DJI_0007_001-005.MP4_snapshot_01.25_[2022.08.11_15.30.41].jpg
2
bb (2001.3, 1617.88) (2764.28, 1510.14)
있음
2
bb (2773.11, 1510.14) (3385.97, 1435.96)
있음
2
bb (577.79, 1794.49) (1709.89, 1646.13)
있음
DJI_0007_001-005.MP4_snapshot_01.28_[2022.08.11_15.30.51].jpg
2
bb (388.81, 1697.35) (1427.31, 1693.82)
있음
2
bb (2711.3, 1672.62) (3691.51, 1651.43)
있음
2
bb (1713.42, 1692.05) (2697.17, 1672.62)
있음
DJI_0007_001-005.MP4_snapshot_01.31_[2022.08.11_15.30.56].jpg
2
bb (2117.87, 1667.33) (3103.38, 1676.16)
있음
2
bb (3175.79, 1683.22) (3840.0, 1672.92)
있음
2
bb (0.0, 1651.53) (828.58, 1658.5)
있음
4
bb [1116.46 1660.26] [2105.51 1660.26]
없음 일반형
DJI_0007_001-005.MP4_snapshot_01.34_[2022.08.11_15.31.00].jpg
2
bb (1983.64, 1614.34) (2884.38, 1628.47)
있음
2
bb (0.0, 1588.22) (729.68, 1603.74)
있음
2
bb (1019.33, 1612.58) (1962.45, 1614.34)
있음
2
bb (2900.28, 1630.24) (3840.0, 1630.24)
있음
DJI_0007_001-005.MP4_snapshot_01.36_[2022.08.11_15.31.04].jpg
2
bb (510.68, 1564.89) (1409.64, 1587.85)
있음
2
bb (2351.0, 15

4
bb [ 500.08 1700.88] [1508.55 1669.09]
없음 일반형
4
bb [2419.88 1639.07] [3209.35 1616.11]
없음 일반형
DJI_0003_002.MP4_snapshot_01.32_[2022.08.22_10.33.44].jpg
2
bb (2790.77, 1534.87) (3377.13, 1504.84)
있음
2
bb (1452.03, 1186.93) (1755.81, 1185.17)
있음
4
bb [   0.  1667.3] [ 780.9  1688.52]
없음 일반형
4
bb [2033.1 1632. ] [2810.2  1594.91]
없음 일반형
4
bb [ 780.9  1688.52] [1748.75 1642.6 ]
없음 일반형
DJI_0003_002.MP4_snapshot_01.33_[2022.08.22_10.33.45].jpg
2
bb (787.96, 1564.89) (1515.61, 1533.1)
있음
2
bb (2004.84, 1185.17) (2276.82, 1188.7)
있음
4
bb [1522.68 1654.96] [2409.29 1594.91]
없음 일반형
4
bb [2414.58 1603.74] [3099.85 1568.42]
없음 일반형
4
bb [1644.5 1217. ] [1990.71 1204.6 ]
없음 일반형
DJI_0003_002.MP4_snapshot_01.57_[2022.08.22_10.34.09].jpg
2
bb (2428.71, 1616.11) (3253.5, 1612.58)
있음
2
bb (1333.7, 1204.6) (1667.5, 1199.3)
있음
4
bb [1453.8  1737.97] [2460.5  1688.52]
없음 일반형
4
bb [ 328.76 1759.17] [1450.27 1727.38]
없음 일반형
4
bb [1655.14 1224.02] [2038.4 1213.4]
없음 일반형
DJI_0003_002.MP4_snapshot_01.58_[2022.

In [5]:
datasets

{'occupied': {'1': {'label': '있음',
   'src': 'DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg',
   'b1': (2229.14, 1681.4655590057796),
   'b2': (2970.92, 1692.0555590057795),
   'b3': (2970.92, 1197.4851656339156),
   'b4': (2229.14, 1186.8951656339157)},
  '2': {'label': '있음',
   'src': 'DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg',
   'b1': (1210.07, 1606.7477891900717),
   'b2': (1999.54, 1610.2777891900716),
   'b3': (1999.54, 1083.9591945895936),
   'b4': (1210.07, 1080.4291945895939)},
  '3': {'label': '있음',
   'src': 'DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg',
   'b1': (-0.0, 1576.8697588717432),
   'b2': (664.33, 1606.6597588717434),
   'b3': (664.33, 1163.3280330601215),
   'b4': (-0.0, 1133.5380330601213)},
  '4': {'label': '있음',
   'src': 'DJI_0001_001.MP4_snapshot_00.18_[2022.08.22_15.41.09].jpg',
   'b1': (1854.71, 1606.0439254770401),
   'b2': (2584.13, 1600.7439254770402),
   'b3': (2584.13, 1114.4510889634385),
   'b4': (1854.71

In [6]:
len(datasets['occupied']), len(datasets['empty']), len(datasets['obstacle'])

(7895, 4958, 348)

In [7]:
len(datasets['occupied']) + len(datasets['empty']) + len(datasets['obstacle'])

13201

In [22]:
dataset_name = 'total_20220724_20220818_update'

FOLDER_NAME = dataset_name + '_generated'
DATASET_ROOT = "/root/share/datasets/"
DATASET_PATH = DATASET_ROOT + FOLDER_NAME
EMPTY_PATH = DATASET_PATH + "/Empty/" # 없음 patch 저장 폴더
OCCUPIED_PATH = DATASET_PATH + "/Occupied/" # 있음 patch 저장 폴더
EMPTY_TYPE_PATH = DATASET_PATH + "/EmptyType"
OBSTACLE_PATH = DATASET_PATH + "/Obstacle"

# 기존 폴더 삭제
if os.path.exists(DATASET_PATH):
    shutil.rmtree(DATASET_PATH)

# 폴더가 없는 경우 생성
if not os.path.isdir(DATASET_PATH):
    os.makedirs(DATASET_PATH)
if not os.path.isdir(EMPTY_PATH):
    os.makedirs(EMPTY_PATH)
if not os.path.isdir(OCCUPIED_PATH):
    os.makedirs(OCCUPIED_PATH)
if not os.path.isdir(EMPTY_TYPE_PATH):
    os.makedirs(EMPTY_TYPE_PATH)
if not os.path.isdir(OBSTACLE_PATH):
    os.makedirs(OBSTACLE_PATH)
    
# Open label txt file 
label_file_all = open(DATASET_PATH + '/'+ dataset_name + '_all.txt', 'w')
label_file_occupied = open(DATASET_PATH + '/' + dataset_name + '_occupied.txt', 'w')
label_file_empty = open(DATASET_PATH + '/' + dataset_name + '_empty.txt', 'w')
label_file_empty_type = open(DATASET_PATH + '/' + dataset_name + '_empty_type.txt', 'w')
label_file_obstacle = open(DATASET_PATH + '/' + dataset_name + '_obstacle.txt', 'w')
label_file_total = open(DATASET_PATH + '/' + dataset_name + '_total.txt', 'w')

In [27]:
for key in datasets['occupied'].keys():
    print(key)
    dataset = datasets['occupied']
    data = dataset[key] # label src b1 b2 b3 b4
    label = data['label']
    src = data['src']
    bb_point = np.array([[data['b1'][0], data['b1'][1]],
                         [data['b2'][0], data['b2'][1]],
                         [data['b3'][0], data['b3'][1]],
                         [data['b4'][0], data['b4'][1]]], dtype=np.float32)
    # source image
    img_path = os.path.join(dataset_name, 'images', src)
    print(img_root)
    src = cv2.imread(img_path, cv2.IMREAD_COLOR)

1
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
2
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
3
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
4
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
5
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
6
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
7
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
8
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
9
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
10
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
11
total_20220724_2

84
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
85
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
86
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
87
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
88
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
89
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
90
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
91
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
92
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
93
total_20220724_20220818_update/images/DJI_0001_001.MP4_snapshot_00.15_[2022.08.22_15.40.57].jpg
94
total_2

KeyboardInterrupt: 

In [12]:
for key in datasets['empty'].keys():
    print(key)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
for key in datasets['obstacle'].keys():
    print(key)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
